#Creación Grafo de Conocimiento Darnel

### Instalación librerias

In [ ]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from neo4j import GraphDatabase

###Conexión a Neo4j

In [ ]:
uri = "neo4j+s://76838788.databases.neo4j.io"
user = "neo4j"
password = "urfpim_kxVWH5LgUxt0M1NcEVRhgyPTd9W5ZXjoRQMQ"
driver = GraphDatabase.driver(uri, auth=(user, password))

#Lectura, exploración y procesamiento de datos

In [ ]:
KB = pd.read_csv('./KB.csv', delimiter=';')
nombres = pd.read_csv('./nombres.csv', delimiter=';')

In [ ]:
KB = KB.rename(columns={"Descripción Problema - Producción": "descripcion_problema_produccion", 
                   "Localización": "localizacion", 
                   "Fecha Generación": "fecha_generacion",
                   "Inicio Intervención": "inicio_intervencion",
                   "Equipo": "equipo",
                   "Fin Intervención": "fin_intervencion",
                   "Descripción Problema - Mantenimiento": "descripcion_problema_mantenimiento",
                   "Causa": "causa",
                   "Acción de Cierre": "accion_cierre",
                   "Especialidad Requerida": "especialidad_requerida",
                   "Repuesto Utilizado": "repuesto_utilizado",
                   "Descripción Repuesto": "descripcion_repuesto",
                   "Cantidad": "cantidad"})

In [ ]:
KB.columns

Index(['BUSINESS_UNIT', 'OT', 'localizacion', 'equipo',
       'descripcion_problema_produccion', 'fecha_generacion',
       'inicio_intervencion', 'fin_intervencion',
       'descripcion_problema_mantenimiento', 'causa', 'accion_cierre',
       'especialidad_requerida', 'repuesto_utilizado', 'descripcion_repuesto',
       'cantidad'],
      dtype='object')

In [ ]:
KB[['linea', 'maquina']] = KB['localizacion'].str.split('-', n=-1, expand=True)
KB['descripcion_problema_produccion'] = KB['descripcion_problema_produccion'].str.replace('\'','')
KB['descripcion_problema_mantenimiento'] = KB['descripcion_problema_mantenimiento'].str.replace('\'','')
KB['causa'] = KB['causa'].str.replace('\'','')
KB['accion_cierre'] = KB['accion_cierre'].str.replace('\'','')
KB['especialidad_requerida'] = KB['especialidad_requerida'].str.replace('\'','')
KB['repuesto_utilizado'] = KB['repuesto_utilizado'].str.replace('\'','')
KB['descripcion_repuesto'] = KB['descripcion_repuesto'].str.replace('\'','')

In [ ]:
nombres[['linea', 'maquina']] = nombres['TREE_NODE'].str.split('-', n=-1, expand=True)
nombres = nombres [['linea', 'maquina', 'AJ_DESCR150', 'AJ_TIPO_LOCA_ID']]
nombres['maquina'] = nombres['maquina'].fillna('')

<ipython-input-8-8ec2516f81a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nombres['maquina'] = nombres['maquina'].fillna('')


In [ ]:
nombres

,linea,maquina,AJ_DESCR150,AJ_TIPO_LOCA_ID
0,ESPTF04,,LINEA DE TERMOFORMADO 4,LINEA
1,ESPTF04,DE01,DESEMBOBINADOR,PADRE
2,ESPTF04,DE01ME01,MOTOR DESEMBOBINADOR,HIJO N1
3,ESPTF04,DE01RD01,REDUCTOR DESEMBOBINADOR,HIJO N1
4,ESPTF04,FM01,FORMADORA,PADRE
...,...,...,...,...
558,ESPTF16,TQ01RD03,REDUCTOR LEFT TROQUEL TROQ,HIJO N1
559,ESPTF16,TQ01RD04,REDUCTOR RIGH TROQUEL TROQ,HIJO N1
560,ESPTF16,TQ01TE01,CAJA DE PULSADORES TROQUEL TROQ,HIJO N1
561,ESPTF16,TQ01TE02,TABLERO DE FUERZA TABLERO DE FUERZA TROQ,HIJO N1


In [ ]:
KB

,BUSINESS_UNIT,OT,localizacion,equipo,descripcion_problema_produccion,fecha_generacion,inicio_intervencion,fin_intervencion,descripcion_problema_mantenimiento,causa,accion_cierre,especialidad_requerida,repuesto_utilizado,descripcion_repuesto,cantidad,linea,maquina
0,AJM01,110199,ESPTF13-HF01ME01,618,Favor revisar falla de comunic,2012-09-18 13:58:15.000,2012-09-16 07:30:00.000,2012-09-16 07:45:00.000,Favor revisar falla de comunicacion en formado...,falla comunicacion,Se resetea ballerina y microdan arranca maquina.,ELE,,NaN,0.0000,ESPTF13,HF01ME01
1,AJM01,110208,ESPTF10-DE01,468,"Favor revisar desembobinador,",2012-09-18 14:39:50.000,2012-09-17 10:00:00.000,2012-09-17 11:20:00.000,"Favor revisar desembobinador, se apaga la...",DRIVE DEL DESEMBOBINADOR NO PARA,SE REVISA Y SE INSTALA RELEVO A 110V PARA PARA...,ELE,,NaN,0.0000,ESPTF10,DE01
2,AJM01,110213,ESPTF10-FM01,483,Favor revisar termoformadora y,2012-09-18 15:11:29.000,2012-09-17 18:30:00.000,2012-09-17 20:10:00.000,"Favor revisar termoformadora y troquel, no pre...",PRESOSTETO EN 20PSI,"SE REVISA Y SE ENCUENTRA PRESOSTETO EN 20PSI, ...",ELE,,NaN,0.0000,ESPTF10,FM01
3,AJM01,110216,ESPTF06-TQ01,457,BRAZOS NO BAJAN BIEN,2012-09-18 15:27:52.000,2012-09-17 21:00:00.000,2012-09-17 22:00:00.000,BRAZOS NO BAJAN BIEN,NO CORTA,SE REVISA SE ENCUENTRA RODAMIENTO SUELTO EN LA...,TURNO,,NaN,0.0000,ESPTF06,TQ01
4,AJM01,110217,ESPTF14-HF01,657,SE CAYO PARTE TRASERA DEL HORN,2012-09-18 15:31:52.000,2012-09-18 14:00:00.000,2012-09-18 16:00:00.000,SE CAYO PARTE TRASERA DEL HORNO INFERIOR ZONAS...,RUPTURA DE TORNILLO DE LA PARTE TRASERA DEL HO...,"SE REVISA, SE ADICIONAN TORNILLOS.",MEC,,NaN,0.0000,ESPTF14,HF01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24381,AJM01,99904,ESPTF10-FM01SR01,490,"Favor revisar, se está apagand",2012-01-30 09:22:25.000,2012-01-30 07:45:00.000,2012-01-30 11:00:00.000,"Favor revisar, se está apagando el térmico de ...",SE APAGAN BOMBAS MOKON.,SE REVSIAN CONSUMOS DE LAS BOMBA Y SE ENCUENTR...,ELE,,NaN,0.0000,ESPTF10,FM01SR01
24382,AJM01,99910,ESPTF16-HF01,760,REVISAR ZONA 19 HORNO INF.NO C,2012-01-30 11:30:45.000,2012-01-30 11:00:00.000,2012-01-30 13:00:00.000,REVISAR ZONA 19 HORNO INF.NO CALIENTA.,SE REVSIA Y SE ENCUENTRA FUSIBLE ABIERTO Y SSR...,"SE CAMBIO SSR Y FUSIBLE, SE HABILITA MAQUINA.",ELE,Z9813004002,RELE DE ESTADO SOLIDO MARCA O,10.000,ESPTF16,HF01
24383,AJM01,99967,ESPTF12-TQ01,594,REVISAR FUGA DE AIRE VALVULAS,2012-01-31 12:09:57.000,2012-01-31 16:00:00.000,2012-01-31 17:00:00.000,REVISAR FUGA DE AIRE VALVULAS DE L0S MARCADORES.,SE SUELTAN CILINDROS DE LAS RUEDAS DEL CANOPY.,SE AJUSTAN CILINDROS CON TORNILLERÍA NUEVA.,ELE,,NaN,0.0000,ESPTF12,TQ01
24384,AJM01,99967,ESPTF12-TQ01,594,REVISAR FUGA DE AIRE VALVULAS,2012-01-31 12:09:57.000,2012-01-31 16:00:00.000,2012-01-31 17:00:00.000,REVISAR FUGA DE AIRE VALVULAS DE L0S MARCADORES.,SE SUELTAN CILINDROS DE LAS RUEDAS DEL CANOPY.,SE AJUSTAN CILINDROS CON TORNILLERÍA NUEVA.,ELE,,NaN,0.0000,ESPTF12,TQ01


## Creación de nodos y relaciones

### Lineas

In [ ]:
lineas = nombres[nombres['AJ_TIPO_LOCA_ID'].str.contains("LINEA")]
lineas

,linea,maquina,AJ_DESCR150,AJ_TIPO_LOCA_ID
0,ESPTF04,,LINEA DE TERMOFORMADO 4,LINEA
40,ESPTF05,,LINEA DE TERMOFORMADO 5,LINEA
80,ESPTF06,,LINEA DE TERMOFORMADO 6,LINEA
98,ESPTF07,,LINEA DE TERMOFORMADO 7,LINEA
116,ESPTF10,,LINEA DE TERMOFORMADO 10,LINEA
187,ESPTF11,,LINEA DE TERMOFORMADO 11,LINEA
227,ESPTF12,,LINEA DE TERMOFORMADO 12,LINEA
277,ESPTF13,,LINEA DE TERMOFORMADO 13,LINEA
326,ESPTF14,,LINEA DE TERMOFORMADO 14,LINEA
415,ESPTF15,,LINEA DE TERMOFORMADO 15,LINEA


In [ ]:
def add_line(linea, nombre):
  query = 'CREATE (l1:Linea { id : \''+ linea.strip() +'\', nombre: \''+ nombre +'\'  }) RETURN l1'
  return query
for linea in lineas.iterrows():
  with driver.session() as session:
    info = session.run(add_line(linea[1]['linea'], linea[1]['AJ_DESCR150']))

### Maquinas

In [ ]:
maquinas = nombres[~nombres['AJ_TIPO_LOCA_ID'].str.contains("LINEA")]
maquinas = maquinas.drop_duplicates(subset=['maquina', 'AJ_TIPO_LOCA_ID'])
maquinas

,linea,maquina,AJ_DESCR150,AJ_TIPO_LOCA_ID
1,ESPTF04,DE01,DESEMBOBINADOR,PADRE
2,ESPTF04,DE01ME01,MOTOR DESEMBOBINADOR,HIJO N1
3,ESPTF04,DE01RD01,REDUCTOR DESEMBOBINADOR,HIJO N1
4,ESPTF04,FM01,FORMADORA,PADRE
5,ESPTF04,FM01BV01,BOMBA DE VACIO FORMADORA,HIJO N1
...,...,...,...,...
526,ESPTF16,FM01SR02BC02,BOMBA 2 FRIGEL FORM,HIJO N2
527,ESPTF16,FM01SR02ME01,MOTOR 1 FRIGEL FORM,HIJO N2
528,ESPTF16,FM01SR02ME02,MOTOR 2 FRIGEL FORM,HIJO N2
529,ESPTF16,FM01SR02TE01,PANEL DE CONTROL FRIGEL FORM,HIJO N2


In [ ]:
def add_machine(maquina, nombre, tipo):
  query = 'CREATE (l1:Maquina { id : \''+ maquina +'\', nombre: \''+ nombre +'\', tipo : \''+ tipo +'\'  }) RETURN l1'
  return query
for maquina in maquinas.iterrows():
  with driver.session() as session:
    info = session.run(add_machine(maquina[1]['maquina'], maquina[1]['AJ_DESCR150'], maquina[1]['AJ_TIPO_LOCA_ID']))

### Relacion Linea-Maquina

In [ ]:
relaciones = KB.groupby(['linea','maquina']).size().reset_index()
relaciones

,linea,maquina,0
0,ESPTF04,DE01,22
1,ESPTF04,DE01ME01,8
2,ESPTF04,FM01,543
3,ESPTF04,FM01BV01,38
4,ESPTF04,FM01BV01ME01,2
...,...,...,...
289,ESPTF16,TQ01ME03,11
290,ESPTF16,TQ01RD03,17
291,ESPTF16,TQ01RD04,8
292,ESPTF16,TQ01TE02,2


In [ ]:
def add_relation(linea, maquina, cuenta):
  query = 'MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = \''+ linea +'\' AND b.id = \''+ maquina +'\' CREATE (a)-[r:TIENE {count: '+ str(cuenta) +'}]->(b) RETURN type(r), r.name'
  return query
for relacion in relaciones.iterrows():
  with driver.session() as session:
    print(relacion[0], add_relation(relacion[1]['linea'], relacion[1]['maquina'], relacion[1][0]))
    info = session.run(add_relation(relacion[1]['linea'], relacion[1]['maquina'], relacion[1][0]))

0 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'DE01' CREATE (a)-[r:TIENE {count: 22}]->(b) RETURN type(r), r.name
1 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'DE01ME01' CREATE (a)-[r:TIENE {count: 8}]->(b) RETURN type(r), r.name
2 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'FM01' CREATE (a)-[r:TIENE {count: 543}]->(b) RETURN type(r), r.name
3 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'FM01BV01' CREATE (a)-[r:TIENE {count: 38}]->(b) RETURN type(r), r.name
4 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'FM01BV01ME01' CREATE (a)-[r:TIENE {count: 2}]->(b) RETURN type(r), r.name
5 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'FM01ME01' CREATE (a)-[r:TIENE {count: 105}]->(b) RETURN type(r), r.name
6 MATCH (a:Linea), (b:Maquina) WHERE trim(a.id) = 'ESPTF04' AND b.id = 'FM01ME02' CREATE (a)-[r:TIENE {count: 49}]->(b) RETURN type(r), r.name
7 M

### Equipos

In [ ]:
my_array = KB['equipo'].unique()
equipos = pd.DataFrame(my_array, columns = ['equipo'])
equipos

,equipo
0,618
1,468
2,483
3,457
4,657
...,...
288,751
289,651
290,759
291,424


In [ ]:
def add_equipment(equipo):
  query = 'CREATE (l1:Equipo { id : '+ str(equipo) +' }) RETURN l1'
  return query
for equipo in equipos.iterrows():
  with driver.session() as session:
    info = session.run(add_equipment(equipo[1]['equipo']))

### Relacion Maquina-Equipo

In [ ]:
relaciones = KB.groupby(['maquina','equipo']).size().reset_index().drop_duplicates(subset=['maquina', 'equipo'])
relaciones

,maquina,equipo,0
0,DE01,376,22
1,DE01,411,27
2,DE01,446,32
3,DE01,458,18
4,DE01,468,85
...,...,...,...
290,TQ01TE03,601,1
291,TQ01TP01,410,28
292,TQ01TP01,776,14
293,TQ02,3490,43


In [ ]:
def add_relation(maquina, equipo, cuenta):
  query = 'MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = \''+ maquina +'\' AND b.id = '+ str(equipo) +' CREATE (a)-[r:TIENE {count: '+ str(cuenta) +'}]->(b) RETURN type(r), r.name'
  return query
for relacion in relaciones.iterrows():
  with driver.session() as session:
    print(relacion[0], add_relation(relacion[1]['maquina'], relacion[1]['equipo'], relacion[1][0]))
    info = session.run(add_relation(relacion[1]['maquina'], relacion[1]['equipo'], relacion[1][0]))

0 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 376 CREATE (a)-[r:TIENE {count: 22}]->(b) RETURN type(r), r.name
1 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 411 CREATE (a)-[r:TIENE {count: 27}]->(b) RETURN type(r), r.name
2 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 446 CREATE (a)-[r:TIENE {count: 32}]->(b) RETURN type(r), r.name
3 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 458 CREATE (a)-[r:TIENE {count: 18}]->(b) RETURN type(r), r.name
4 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 468 CREATE (a)-[r:TIENE {count: 85}]->(b) RETURN type(r), r.name
5 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 532 CREATE (a)-[r:TIENE {count: 55}]->(b) RETURN type(r), r.name
6 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND b.id = 567 CREATE (a)-[r:TIENE {count: 176}]->(b) RETURN type(r), r.name
7 MATCH (a:Maquina), (b:Equipo) WHERE trim(a.id) = 'DE01' AND

### Creación Problemas

In [ ]:
problemas = KB.groupby(['equipo','descripcion_problema_produccion', 'causa', 'especialidad_requerida']).size().reset_index()
problemas

,equipo,descripcion_problema_produccion,causa,especialidad_requerida,0
0,376,AMPLIFICADOR DE SEÑAL NO FUNCI,DESAJUSTE EN FOTOCELDAS RODILLOS HALADORES,ELE,1
1,376,CAMBIAR ESPEJO REFEX NO ESTA R,VARIACIÓN DE SEÑAL DE ESPEJO REFLECTOR DEL DES...,TURNO,1
2,376,EL DESEMBOBINADDOR NO HALA LA,ESCOBILLA DEL MOTOR SUELTA,TURNO,3
3,376,FAVOR REVISAR DESEMBOBINADOR N,SE ENCUENTRA VARIADOR EN FALLA NO FUNCIONA.,ELE,1
4,376,Favor revisar el desembobinado,TARJETA DE CONTRO VARIADOR EN FALLA,ELE,1
...,...,...,...,...,...
8104,4196,Fuga de aceite de cilindro inf,Fuga de aceite por un cilindro,MEC,1
8105,4198,Cadena se frena,Sensor de plancha posición superior no sensa,ELE,1
8106,4198,Hace corto una temperatura,Termocupla suelta,ELE,7
8107,4379,Ruedas del Canopy pegadas,Electroválvula de las ruedas del Canopy no actua.,ELE,1


In [ ]:
def add_problem(id, descripcion_problema_produccion, causa, especialidad_requerida, equipo):
  query = 'CREATE (l1:Problema { descripcion: \''+ descripcion_problema_produccion +'\', causa : \''+ causa +'\', especialidad_requerida : \''+ especialidad_requerida +'\', equipo : '+ str(equipo) +'  }) RETURN l1'
  return query
for problema in problemas.iterrows():
  with driver.session() as session:
    info = session.run(add_problem(problema[0], problema[1]['descripcion_problema_produccion'], problema[1]['causa'], problema[1]['especialidad_requerida'], problema[1]['equipo']))

### Creación relación Equipo- Problema

In [ ]:
def add_relation(equipo, cuenta):
  query = 'MATCH (a:Equipo), (b:Problema) WHERE a.id = '+ str(equipo) +' AND b.equipo = '+ str(equipo) +' CREATE (a)-[r:OCURRIO {count: '+ str(cuenta) +'}]->(b) RETURN type(r), r.name'
  return query
for equipo in equipos.iterrows():
  with driver.session() as session:
    print(relacion[0], add_relation(equipo[1]['equipo'], equipo[1][0]))
    info = session.run(add_relation(equipo[1]['equipo'], equipo[1][0]))

294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 618 AND b.equipo = 618 CREATE (a)-[r:OCURRIO {count: 618}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 468 AND b.equipo = 468 CREATE (a)-[r:OCURRIO {count: 468}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 483 AND b.equipo = 483 CREATE (a)-[r:OCURRIO {count: 483}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 457 AND b.equipo = 457 CREATE (a)-[r:OCURRIO {count: 457}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 657 AND b.equipo = 657 CREATE (a)-[r:OCURRIO {count: 657}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 403 AND b.equipo = 403 CREATE (a)-[r:OCURRIO {count: 403}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 414 AND b.equipo = 414 CREATE (a)-[r:OCURRIO {count: 414}]->(b) RETURN type(r), r.name
294 MATCH (a:Equipo), (b:Problema) WHERE a.id = 746 AND

### Creación Soluciones



In [ ]:
soluciones = KB.groupby(['equipo', 'descripcion_problema_produccion','descripcion_problema_mantenimiento','inicio_intervencion', 'fin_intervencion', 'accion_cierre', 'repuesto_utilizado', 'descripcion_repuesto','cantidad']).size().reset_index()
soluciones

,equipo,descripcion_problema_produccion,descripcion_problema_mantenimiento,inicio_intervencion,fin_intervencion,accion_cierre,repuesto_utilizado,descripcion_repuesto,cantidad,0
0,376,Favor revisar el desembobinado,"Favor revisar el desembobinador, el motor se d...",2011-09-22 15:00:00.000,2011-09-22 18:45:00.000,SE CAMBIO VARIADOR Y SE HABILITA MAQUINA,Z9444300016,VARIADOR DE VELOC. 2HP/440VAC,10.000,1
1,376,Revisar desembobinador no avan,Revisar desembobinador no avanza,2014-11-03 13:40:00.000,2014-11-03 15:10:00.000,"SE HACE BORRADO Y SE PROGRAMA NUEVAMENTE, SE D...",Z0403800017,MANGUERA FLEXIBLE EN POLIURETA,20.000,1
2,376,Revisar desembobinador no avan,Revisar desembobinador no avanza,2014-11-03 13:40:00.000,2014-11-03 15:10:00.000,"SE HACE BORRADO Y SE PROGRAMA NUEVAMENTE, SE D...",Z0500053203,TORNILLO BRISTOL CON CABEZA .,20.000,1
3,376,Revisar desembobinador no avan,Revisar desembobinador no avanza,2014-11-03 13:40:00.000,2014-11-03 15:10:00.000,"SE HACE BORRADO Y SE PROGRAMA NUEVAMENTE, SE D...",Z0500800604,TORNILLO CABEZA HEXAGONAL - NC,60.000,1
4,376,Revisar desembobinador no avan,Revisar desembobinador no avanza,2014-11-03 13:40:00.000,2014-11-03 15:10:00.000,"SE HACE BORRADO Y SE PROGRAMA NUEVAMENTE, SE D...",Z0503300006,ARANDELA PLANA 3/8``,60.000,1
...,...,...,...,...,...,...,...,...,...,...
9330,4198,Hace corto una temperatura,Hace corto una temperatura,2023-03-11 19:41:40.880,2023-03-11 19:42:58.000,Se aprieta termocupla,Z0903200400,UNION PARA CADENA SENCILLA # 4,10.000,1
9331,4198,Hace corto una temperatura,Hace corto una temperatura,2023-03-11 19:41:40.880,2023-03-11 19:42:58.000,Se aprieta termocupla,Z9082000006,TERMINAL PLATEADO CODIGO: 6179,100.000,1
9332,4198,Hace corto una temperatura,Hace corto una temperatura,2023-03-11 19:41:40.880,2023-03-11 19:42:58.000,Se aprieta termocupla,Z9090124350,FUSIBLE CLASE J-TIME DELAY-600,30.000,1
9333,4198,Hace corto una temperatura,Hace corto una temperatura,2023-03-11 19:41:40.880,2023-03-11 19:42:58.000,Se aprieta termocupla,Z9421140134,TYPE J THERMOCOUPLE P/N MMJ51U,10.000,1


In [ ]:
def add_soluciones(id, descripcion_problema_produccion, descripcion_problema_mantenimiento, inicio_intervencion, equipo, fin_intervencion, accion_cierre,repuesto_utilizado, descripcion_repuesto, cantidad):
  query = 'CREATE (l1:Solucion { id : \''+ str(id) +'\', descripcion: \''+ descripcion_problema_produccion +'\', mantenimiento : \''+ descripcion_problema_mantenimiento +'\', inicio_intervencion : \''+ inicio_intervencion +'\', equipo : '+ str(equipo) +', fin_intervencion : \''+ fin_intervencion +'\', accion_cierre : \''+ accion_cierre +'\', descripcion_repuesto : \''+ descripcion_repuesto +'\', cantidad : \''+ cantidad +'\' }) RETURN l1'
  return query
for solucion in soluciones.iterrows():
  with driver.session() as session:
    print(solucion[0], add_soluciones(solucion[0], solucion[1]['descripcion_problema_produccion'], solucion[1]['descripcion_problema_mantenimiento'], solucion[1]['inicio_intervencion'], solucion[1]['equipo'], solucion[1]['fin_intervencion'], solucion[1]['accion_cierre'], solucion[1]['repuesto_utilizado'], solucion[1]['descripcion_repuesto'], solucion[1]['cantidad']))
    info = session.run(add_soluciones(solucion[0], solucion[1]['descripcion_problema_produccion'], solucion[1]['descripcion_problema_mantenimiento'], solucion[1]['inicio_intervencion'], solucion[1]['equipo'], solucion[1]['fin_intervencion'], solucion[1]['accion_cierre'], solucion[1]['repuesto_utilizado'], solucion[1]['descripcion_repuesto'], solucion[1]['cantidad']))

Se truncaron las últimas líneas 5000 del resultado de transmisión.
4335 CREATE (l1:Solucion { id : '4335', descripcion: 'Se revienta cadena lado izquie', mantenimiento : 'Se revienta cadena lado izquierdo', inicio_intervencion : '2021-10-01 10:24:00.000', equipo : 570, fin_intervencion : '2021-10-01 11:48:00.000', accion_cierre : 'Se desatasca cadena que estaba atascada con materia espumado', descripcion_repuesto : 'CABLE ZERO-FLAME SC 350  1x4', cantidad : '10.000' }) RETURN l1
4336 CREATE (l1:Solucion { id : '4336', descripcion: 'Se revienta cadena lado izquie', mantenimiento : 'Se revienta cadena lado izquierdo', inicio_intervencion : '2021-10-01 10:24:00.000', equipo : 570, fin_intervencion : '2021-10-01 11:48:00.000', accion_cierre : 'Se desatasca cadena que estaba atascada con materia espumado', descripcion_repuesto : 'CABLE ZERO-FLAME SC 350  1x4', cantidad : '150.000' }) RETURN l1
4337 CREATE (l1:Solucion { id : '4337', descripcion: 'Se revienta cadena lado izquie', mantenimien

### Creación Relación Problema-Solución

In [ ]:
def add_relation(equipo, problema, cuenta):
  query = 'MATCH (a:Problema), (b:Solucion) WHERE a.equipo = '+ str(equipo) +' AND b.equipo = '+ str(equipo) +' AND a.descripcion = \''+ problema +'\' AND b.descripcion = \''+ problema +'\'  CREATE (a)-[r:SE_RESOLVIO {count: '+ str(cuenta) +'}]->(b) RETURN type(r), r.name'
  return query
for problema in problemas.iterrows():
  with driver.session() as session:
    print(problema[0], add_relation(problema[1]['equipo'], problema[1]['descripcion_problema_produccion'], problema[1][0]))
    info = session.run(add_relation(problema[1]['equipo'], problema[1]['descripcion_problema_produccion'], problema[1][0]))

Se truncaron las últimas líneas 5000 del resultado de transmisión.
3109 MATCH (a:Problema), (b:Solucion) WHERE a.equipo = 547 AND b.equipo = 547 AND a.descripcion = 'Favor revisar zonas 11 y 14 ho' AND b.descripcion = 'Favor revisar zonas 11 y 14 ho'  CREATE (a)-[r:SE_RESOLVIO {count: 1}]->(b) RETURN type(r), r.name
3110 MATCH (a:Problema), (b:Solucion) WHERE a.equipo = 547 AND b.equipo = 547 AND a.descripcion = 'HORNO ELECTRICO NO CIERRA.' AND b.descripcion = 'HORNO ELECTRICO NO CIERRA.'  CREATE (a)-[r:SE_RESOLVIO {count: 1}]->(b) RETURN type(r), r.name
3111 MATCH (a:Problema), (b:Solucion) WHERE a.equipo = 547 AND b.equipo = 547 AND a.descripcion = 'HORNO NO CALIENTA.' AND b.descripcion = 'HORNO NO CALIENTA.'  CREATE (a)-[r:SE_RESOLVIO {count: 1}]->(b) RETURN type(r), r.name
3112 MATCH (a:Problema), (b:Solucion) WHERE a.equipo = 547 AND b.equipo = 547 AND a.descripcion = 'Horno inferior parte delantera' AND b.descripcion = 'Horno inferior parte delantera'  CREATE (a)-[r:SE_RESOLVIO {